# Hyperparameter tuning

los modelos tienen dos tipos de parámetros:

* Parámetros entrenables o parámetros del modelo: aquellos que el algoritmo aprende a partir de los datos, por ejemplo en una regresión coeficientes o pesos, un intercepto, o en un árbol las feature importances...

* Hiperparámetros: se configuran antes del entrenamiento, afecta a la forma en la que el algoritmo aprende. Por ejemplo el max_depth de un árbol de decisión o el max_iter de LogisticRegression o n_neighbors de KNN. Controlan la complejidad del modelo y cómo aprende.

En Scikit Learn tenemos clases para probar combinaciones de hyperparameters de forma automática:

* GridSearchCV
* RandomizedSearchCV

## GridSearchCV

In [11]:
import numpy as np
np.arange(1, 21)

array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
       18, 19, 20])

In [13]:
list(range(1,21))

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20]

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.datasets import load_iris
from sklearn.tree import DecisionTreeClassifier
import numpy as np
X, y = load_iris(return_X_y=True)

model = DecisionTreeClassifier(random_state=42)

params = {
    'max_depth': np.arange(1,10),
    'min_samples_split': np.arange(2,10),
    'criterion': ['gini', 'entropy']
}

grid = GridSearchCV(model, params, scoring='accuracy', verbose=1) # cv=5 por defecto
grid.fit(X, y)

Fitting 5 folds for each of 380 candidates, totalling 1900 fits


GridSearchCV(estimator=DecisionTreeClassifier(random_state=42),
             param_grid={'max_depth': array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
       18, 19, 20]),
                         'min_samples_split': array([ 2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17, 18,
       19, 20])},
             scoring='accuracy', verbose=1)

In [17]:
print('best params:', grid.best_params_)
print('best params:', grid.best_score_)

best params: {'max_depth': np.int64(3), 'min_samples_split': np.int64(2)}
best params: 0.9733333333333334


In [19]:
# Opción 1: usar el grid directamente, ya se puede usar igual que un modelo, con la función predict:
# ya está entrenado
grid.predict([[5.1, 3.5, 1.4, 0.2]])

array([0])

In [21]:
# Opción 2: extraer el modelo del grid
# ya está entrenado
best_model = grid.best_estimator_
print(type(best_model))
print(best_model.predict([[5.1, 3.5, 1.4, 0.2]]))

<class 'sklearn.tree._classes.DecisionTreeClassifier'>
[0]


In [23]:
# Opción 3: crear otro algoritmo utilizando los best param por ejemplo para hacer otro experimento o entrenar ya con todo el dataset
# habría que entrenarlo
model_2 = DecisionTreeClassifier(**grid.best_params_)
model_2.fit(X, y)
print(model_2.predict([[5.1, 3.5, 1.4, 0.2]]))

[0]


In [26]:
# si queremos sacar los params de un modelo ya existente
print(model_2.get_params())

{'ccp_alpha': 0.0, 'class_weight': None, 'criterion': 'gini', 'max_depth': np.int64(3), 'max_features': None, 'max_leaf_nodes': None, 'min_impurity_decrease': 0.0, 'min_samples_leaf': 1, 'min_samples_split': np.int64(2), 'min_weight_fraction_leaf': 0.0, 'monotonic_cst': None, 'random_state': None, 'splitter': 'best'}


## GridSearchCV con múltiples métricas

Al igual que en validación cruzada con cross_validate podemos usar múltiples métricas en el GridSearchCV.

Obligatorio usar el parámetro refit para indicar qué métrica debe usarse cuando ya se han encontrado los mejores parámetros.

In [38]:
X, y = load_iris(return_X_y=True)

model = DecisionTreeClassifier(random_state=42)

params = {
    'max_depth': np.arange(1,10),
    'min_samples_split': np.arange(2,10),
    'criterion': ['gini', 'entropy']
}

grid = GridSearchCV(model, params, scoring=['accuracy', 'f1_macro', 'roc_auc_ovr'], refit='f1_macro', verbose=1) # cv=5 por defecto
grid.fit(X, y)

Fitting 5 folds for each of 144 candidates, totalling 720 fits


GridSearchCV(estimator=DecisionTreeClassifier(random_state=42),
             param_grid={'criterion': ['gini', 'entropy'],
                         'max_depth': array([1, 2, 3, 4, 5, 6, 7, 8, 9]),
                         'min_samples_split': array([2, 3, 4, 5, 6, 7, 8, 9])},
             refit='f1_macro', scoring=['accuracy', 'f1_macro', 'roc_auc_ovr'],
             verbose=1)

In [ ]:
# VER LOS RESULTADOS DEL ENTRENAMIENTO DEL GRIDSEARCH CON TODAS LAS COMBINACIONES:
import pandas as pd
pd.DataFrame(grid.cv_results_).head(2)

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_criterion,param_max_depth,param_min_samples_split,params,split0_test_accuracy,split1_test_accuracy,...,std_test_f1_macro,rank_test_f1_macro,split0_test_roc_auc_ovr,split1_test_roc_auc_ovr,split2_test_roc_auc_ovr,split3_test_roc_auc_ovr,split4_test_roc_auc_ovr,mean_test_roc_auc_ovr,std_test_roc_auc_ovr,rank_test_roc_auc_ovr
0,0.002020,0.000627,0.009112,0.001520,gini,1,2,"{'criterion': 'gini', 'max_depth': 1, 'min_sam...",0.666667,0.666667,...,0.0,129,0.833333,0.833333,0.833333,0.833333,0.833333,0.833333,0.0,129
1,0.001603,0.000446,0.011623,0.005581,gini,1,3,"{'criterion': 'gini', 'max_depth': 1, 'min_sam...",0.666667,0.666667,...,0.0,129,0.833333,0.833333,0.833333,0.833333,0.833333,0.833333,0.0,129


## RandomizedSearchCV

Alternativa a GridSearchCV para reducir el tiempo de cómputo.

Se seleccionan combinaciones de forma aleatoria según distribuciones predefinidas.

In [53]:
from sklearn.model_selection import RandomizedSearchCV

model = DecisionTreeClassifier(random_state=42)

params = {
    'max_depth': np.arange(1,20),
    'min_samples_split': np.arange(2,20),
    'criterion': ['gini', 'entropy']
}

grid = RandomizedSearchCV(model, params, n_iter=10, scoring='accuracy', random_state=42, verbose=1, cv=5) # cv=5 por defecto
grid.fit(X, y)

print(grid.best_params_)
print(grid.best_score_)

Fitting 5 folds for each of 10 candidates, totalling 50 fits
{'min_samples_split': np.int64(17), 'max_depth': np.int64(13), 'criterion': 'gini'}
0.9666666666666668
